In [1]:
# tensorflow 환경이어야함.

# 참조
# - https://wikidocs.net/92961

In [2]:
import os
os.chdir(os.getcwd() + '/../../..')

In [6]:
from lib.python.pandas import set_display_option
set_display_option()

import glob
from tqdm import tqdm
import pandas as pd
import numpy as np

In [4]:
# pip install git+https://github.com/haven-jeon/PyKoSpacing.git
import tensorflow as tf
from pykospacing import Spacing
spacing = Spacing()

from soynlp.normalizer import *

# # pip install git+https://github.com/ssut/py-hanspell.git
# from hanspell import spell_checker

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2024-04-07 06:21:49.320340: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 06:21:55.759489: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-07 06:21:55.760885: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/

In [97]:
# load
path = 'crawling/naver_shopping_review/.result/20240324/*'
paths = glob.glob(path)
df = pd.concat([pd.read_parquet(path) for path in tqdm(paths)],axis=0)

# pre-processing
unuse_columns = ['product_id','id','review_page','aidaModifyTime','esModifyTime','imageYn','images',
                 'mallId','mallSeq','matchNvMid','modifyDate','nvMid','pageUrl','registerDate',
                 'title','topicYn','userId','videoYn','videos','mallLogoUrl']
df.drop(columns=unuse_columns, inplace=True)

100%|██████████| 631/631 [00:06<00:00, 90.63it/s] 


In [98]:
d = df[(df.keyword=='유산균') & (df.review_topic=='주제전체') & (df.product_ranking==1)]

In [99]:
d.head(2)

,keyword,product_ranking,product_title,review_topic,review_ranking,buyOption,content,aidaCreateTime,imageCount,mallProductId,mallReviewId,qualityScore,rankScore,starScore,topicCount,topics,videoCount,mallName
0,유산균,1,에스네이처 아쿠아 스쿠알란 수분크림 60ml,주제전체,1,용량: 60ml 본품만,제형- 크림 제형 펴바를때 꾸덕하지 않고 부드럽게 잘 펼쳐짐<br>수분감- 수분보충...,2023-08-06 15:32:40,4,8227910747,4306619333,0.752075,1.1020750000000001,5,1,"[{'topicCode': 'moisture', 'topicName': '보습력',...",0,로하스힐링
1,유산균,1,에스네이처 아쿠아 스쿠알란 수분크림 60ml,주제전체,2,None,다른 제품만 <em>쓰다가 처음 써봤는데</em> 제 피부에 정말 잘 맞아요! 저는...,2022-03-26 15:36:28,3,5015597646,3396996187,0.696892,1.056892,5,4,"[{'topicCode': 'total', 'topicName': '만족도', 's...",0,에스네이처


In [103]:
print(d.content.values[0])
print(d.topics.values[0])

제형- 크림 제형 펴바를때 꾸덕하지 않고 부드럽게 잘 펼쳐짐<br>수분감- 수분보충에 좋은 스쿠알란이 많이 있어 보습에 좋음. 한번만 발라도 충분한 수분을 충전할 수 있음<br>(다만, 세수하고 나오면 인중과 턱이 바로 땡기고 피부가 일어나는 극 건성임에도 너무 더운 요즘 같은 여름에는 과하게 느껴질 때가 있음)<br>향- 아무향 안남<br>마무리감- 조금의 끈적함이 남을 수 있음
[{'topicCode': 'moisture', 'topicName': '보습력', 'startPosition': 96, 'endPosition': 164, 'positiveYn': 'Y', 'reputationScore': 800}]


In [ ]:
import re

PUNCT = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
PUNCT_MAPPING = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', } 
SPECIALS = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}

def clean_text(text):
    for p in PUNCT_MAPPING:
        text = text.replace(p, PUNCT_MAPPING[p])
    # for p in PUNCT:
    #     text = text.replace(p, f' {p} ')
    for s in SPECIALS:
        text = text.replace(s, SPECIALS[s])
    
    patterns = {
        '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' : '',  # E-mail제거
        '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' : '', # URL제거
        '([ㄱ-ㅎㅏ-ㅣ]+)' : '',                                     # 한글 자음, 모음 제거
        '<[^>]*>' : '',                                           # HTML 태그 제거
        '[^\w\s\n]' : '',                                         # 특수기호제거
        '[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]' : '',
        '\n' : '.',
        '[</br>]' : '.',
        '[<br>]' : '',
    }
    for pattern,repl in patterns.items():
        text = re.sub(pattern=pattern, repl=repl, string=text)
        
    #text = text.replace('em','<em>')
    
    return text.strip()

def preprocess_korean_text(sent):
    sent = ' '.join([s if (s.find('<em>')>=0)|(s.find('</em>')>=0) else clean_text(s) for s in sent.split(' ')])
    #sent = clean_text(sent)
    sent = spacing(sent)
    sent = repeat_normalize(sent, num_repeats=1)
    
    return sent

In [ ]:
# sent = df.content.values[0]
# sent.split(' ')

In [ ]:
for i in range(10):
    sent = df.content.values[i]

    print(sent)
    print(preprocess_korean_text(sent))
    print('')